<a href="https://colab.research.google.com/github/axel-sirota/IEEE-CICD/blob/master/module5/TF_Developer_NLP_Module5_Solution_Text_Generation_Words.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Generating Support like responses


Let's first import everything and load the dataset

In [1]:
!pip install textblob 'gensim==4.2.0' 'keras-nlp' swifter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [41]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from keras import preprocessing
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Model, Input
from collections import defaultdict
import keras_nlp
import swifter
import numpy as np
import re
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import time

TRACE = False
embedding_dim = 128
rnn_units = 1024
epochs=15
buffer_size = 2000
batch_size = 256
min_count_words = 5
fraction_corpus = 1
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config) 
  K.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now we will download the dataset and explore it

In [3]:
%%writefile get_data.sh
if [ ! -f twitter.csv ]; then
  wget -O twitter.csv https://www.dropbox.com/s/hjji6x6vx4wno1c/twcs.csv?dl=0
fi

Overwriting get_data.sh


In [4]:
!bash get_data.sh


Let's inspect how this intents csv looks like

In [5]:
! head twitter.csv

tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist you. We would need to get you into a private secured link to further assist.,2,3
2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,,1
3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messages and no one is responding as usual,1,4
4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so that we can further assist you. Just click ‘Message’ at the top of your profile.,3,5
5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6
6,sprintcare,False,Tue Oct 31 21:46:24 +0000 2017,"@115712 Can you please send us a private message, so that I can gain further details about your account?","5,7",8
8,115712,True,Tue Oct 31 21:45:10 +0000 2017,@sprintcare is the worst customer service,"9,6,10",
11,s

In [6]:
def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"(\@\d+)", r" ", text) # remove twitter handle
    text = re.sub(r"(https\:\/\/t.co\/\w+)", r" ", text) # remove end twtter url
    text = re.sub(r"([.,!?])", r" ", text) # remove punctuation
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [7]:
path = './twitter.csv'
twitter = pd.read_csv(path, header=0)
twitter

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0
...,...,...,...,...,...,...,...
2811769,2987947,sprintcare,False,Wed Nov 22 08:43:51 +0000 2017,"@823869 Hey, we'd be happy to look into this f...",NaN,2987948.0
2811770,2987948,823869,True,Wed Nov 22 08:35:16 +0000 2017,@115714 wtf!? I’ve been having really shitty s...,2987947,NaN
2811771,2812240,121673,True,Thu Nov 23 04:13:07 +0000 2017,@143549 @sprintcare You have to go to https://...,NaN,2812239.0
2811772,2987949,AldiUK,False,Wed Nov 22 08:31:24 +0000 2017,"@823870 Sounds delicious, Sarah! 😋 https://t.c...",NaN,2987950.0


In [8]:
twitter[twitter.inbound == False ].author_id.value_counts()

AmazonHelp        169840
AppleSupport      106860
Uber_Support       56270
SpotifyCares       43265
Delta              42253
                   ...  
JackBox              266
OfficeSupport        218
AskDSC               210
CarlsJr              196
HotelTonightCX       152
Name: author_id, Length: 108, dtype: int64

In [9]:
twitter_filtered = twitter[twitter.inbound == False ].groupby('author_id').filter(lambda x: len(x) >= 10000).sample(frac=fraction_corpus).reset_index()
twitter_filtered.author_id.value_counts()

AmazonHelp         169840
AppleSupport       106860
Uber_Support        56270
SpotifyCares        43265
Delta               42253
Tesco               38573
AmericanAir         36764
TMobileHelp         34317
comcastcares        33031
British_Airways     29361
SouthwestAir        28977
VirginTrains        27817
Ask_Spectrum        25860
XboxSupport         24557
sprintcare          22381
hulu_support        21872
sainsburys          19466
GWRHelp             19364
AskPlayStation      19098
ChipotleTweets      18749
VerizonSupport      17966
UPSHelp             17817
ATVIAssist          17650
O2                  16212
Safaricom_Care      16077
idea_cares          15724
AskTarget           13218
AirAsiaSupport      12829
BofA_Help           12683
SW_Help             12231
ArgosHelpers        12174
AskLyft             11809
marksandspencer     11504
AskAmex             11358
MicrosoftHelps      11304
AskPayPal           11298
Morrisons           10126
Name: author_id, dtype: int64

In [10]:
responses = twitter_filtered.reset_index(drop=True).text
responses

0          @340426 Apologies! We currently don't support ...
1          @130645 Hi, please let us know if you have a s...
2          @820651 Sure Ahmed, may we have the booking nu...
3          @223341 I'm sorry to hear that you did not rec...
4          @399801 traffic through this area to keep the ...
                                 ...                        
1050650    @198997 Please don’t provide your order detail...
1050651    @307390 😢 This isn't something we offer to do,...
1050652                      @214106 You are welcome! \n^HSB
1050653    @577396 Hello, I'm sorry to see that you feel ...
1050654       @206586 @824 Haven't forgotten about you. *HKM
Name: text, Length: 1050655, dtype: object

In [11]:
responses[0]

"@340426 Apologies! We currently don't support off line streaming, but we'll express your interest in seeing this option in the future."

In [12]:
processed_responses = responses.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/1050655 [00:00<?, ?it/s]

In [13]:
processed_responses[0]

'apologies we currently don support off line streaming but we ll express your interest in seeing this option in the future'

In [14]:
vocab = set()
counts = defaultdict(int)
for ix, response in processed_responses.iteritems():
    for word in preprocess_text(response, should_join=False):
        counts[word] += 1
        if counts[word] > min_count_words:
          vocab.add(word)
vocab = sorted(vocab)

In [15]:
print(f'{len(vocab)} unique words')

20006 unique words


In [16]:
def get_maximum_review_length(srs):
    maximum = 0
    for response in srs:
        candidate = len(preprocess_text(response, should_join=False))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(processed_responses)

In [17]:
maximum

60

In [18]:

ids_from_words = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)


In [19]:

words_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_words.get_vocabulary(), invert=True, mask_token=None)


In [20]:
def text_from_ids(ids):
  return tf.strings.reduce_join(words_from_ids(ids), axis=-1, separator=' ')


In [21]:
ids = ids_from_words(preprocess_text('Only you can prevent forest fires', should_join=False))
ids

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([12530, 19909,  2688, 13745,  6959,  6795])>

In [22]:
text_from_ids(ids)


<tf.Tensor: shape=(), dtype=string, numpy=b'only you can prevent forest fires'>

## Creating the corpus and tokenizing

In [23]:
X_train, X_test = train_test_split(processed_responses, test_size=0.2, random_state=42)
X_train

651415    really sorry to hear this ang please pop them ...
517143    hi apologize for the inconvenience can you ple...
479923    hi there could you send us dm with your accoun...
192331    yes please shoot me dm so we can dig into this...
929258    thank you for taking the time to provide your ...
                                ...                        
110268    if there is stock available then yes they will...
259178    apologies for the trouble caused we request yo...
131932    we be glad to take look at this with you can y...
671155    we look forward to welcoming you on board your...
121958    not from us there isn dm us any questions you ...
Name: text, Length: 840524, dtype: object

In [24]:
def pad_sequence_of_tokens(x, maxlen, unk_token='[UNK]'):
  if len(x)<maxlen:
    x.extend([unk_token]*(maxlen-len(x)))
  return x

In [25]:

def get_ids_tensor(srs):
  
  processed = srs.swifter.apply(lambda x: pad_sequence_of_tokens(preprocess_text(x, should_join=False), maxlen=maximum)).to_list()
  return tf.squeeze(tf.constant(pad_sequences(ids_from_words(processed), maxlen=maximum, padding='post'), dtype='int32'))



In [26]:
all_ids = get_ids_tensor(srs=X_train.reset_index(drop=True))
all_ids

Pandas Apply:   0%|          | 0/840524 [00:00<?, ?it/s]

<tf.Tensor: shape=(840524, 60), dtype=int32, numpy=
array([[14405, 16678, 18047, ...,     0,     0,     0],
       [ 8132,  1065,  6943, ...,     0,     0,     0],
       [ 8132, 17827,  4099, ...,     0,     0,     0],
       ...,
       [19433,  1811,  7502, ...,     0,     0,     0],
       [19433, 10548,  7003, ...,     0,     0,     0],
       [12228,  7119, 18789, ...,     0,     0,     0]], dtype=int32)>

In [27]:
all_ids.shape

TensorShape([840524, 60])

In [28]:
test_all_ids = get_ids_tensor(srs=X_test)

Pandas Apply:   0%|          | 0/210131 [00:00<?, ?it/s]

In [29]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
test_ids_dataset = tf.data.Dataset.from_tensor_slices(test_all_ids)

In [30]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [31]:
dataset = ids_dataset.map(split_input_target)
test_dataset = test_ids_dataset.map(split_input_target)

In [32]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'really sorry to hear this ang please pop them along to store and our team will put this right [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]'
Target: b'sorry to hear this ang please pop them along to store and our team will put this right [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK]'


In [33]:

shuffled_batched_dataset = (
    dataset
    .shuffle(buffer_size)
    .batch(batch_size=batch_size, drop_remainder=True)
)
shuffled_batched_test_dataset = (
    test_dataset
    .shuffle(buffer_size)
    .batch(batch_size=batch_size, drop_remainder=True)
)

## Doing the train_test split and defining model

In [34]:
class RentalGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.rnn = tf.keras.layers.LSTM(rnn_units,
                                   activation='tanh',
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.rnn.get_initial_state(x)
    output = self.rnn(x, initial_state=states, training=training)
    x = output[0]
    states = output[1:]
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x
  
  def build_graph(self, maxlen):
    x = Input(shape=(maxlen, ))
    return Model(inputs=x, outputs=self.call(x))

In [42]:
model = RentalGenerator(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_words.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [43]:
for input_example_batch, target_example_batch in shuffled_batched_dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(input_example_batch.shape, "# Input: (batch_size, sequence_length)")
    print(example_batch_predictions.shape, "# Output: (batch_size, sequence_length, vocab_size)")

(256, 59) # Input: (batch_size, sequence_length)
(256, 59, 20007) # Output: (batch_size, sequence_length, vocab_size)


In [44]:
model.build_graph(maxlen=maximum)  # Notice the change in outputs!

In [45]:
model.summary()

Model: "rental_generator_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 60, 128)           2560896   
                                                                 
 lstm_1 (LSTM)               [(None, 60, 256),         394240    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
 dense_1 (Dense)             (None, 60, 20007)         5141799   
                                                                 
Total params: 8,096,935
Trainable params: 8,096,935
Non-trainable params: 0
_________________________________________________________________


In [46]:
perplexity = keras_nlp.metrics.Perplexity(from_logits=True, name='perplexity', mask_token_id=0)
model.compile(optimizer='adam', loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=[perplexity])


Notice the non-trainable parameters

In [47]:
history = model.fit(shuffled_batched_dataset, epochs=epochs, workers=5, validation_data=shuffled_batched_test_dataset)


Epoch 1/30
3283/3283 [==============================] - 169s 51ms/step - loss: 1.4808 - perplexity: 116.2160 - val_loss: 1.0822 - val_perplexity: 35.9736
Epoch 2/30
3283/3283 [==============================] - 168s 51ms/step - loss: 1.0017 - perplexity: 27.5907 - val_loss: 0.9493 - val_perplexity: 23.2077
Epoch 3/30
3283/3283 [==============================] - 168s 51ms/step - loss: 0.9121 - perplexity: 20.5090 - val_loss: 0.8959 - val_perplexity: 19.4771
Epoch 4/30
3283/3283 [==============================] - 167s 51ms/step - loss: 0.8668 - perplexity: 17.6517 - val_loss: 0.8679 - val_perplexity: 17.7715
Epoch 5/30
3283/3283 [==============================] - 168s 51ms/step - loss: 0.8385 - perplexity: 16.0653 - val_loss: 0.8498 - val_perplexity: 16.6853
Epoch 6/30
3283/3283 [==============================] - 168s 51ms/step - loss: 0.8186 - perplexity: 15.0390 - val_loss: 0.8387 - val_perplexity: 16.0510
Epoch 7/30
3283/3283 [==============================] - 168s 51ms/step - loss: 0.

KeyboardInterrupt: ignored

## Some plots

In [ ]:
import matplotlib.pyplot as plt
# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=5.0)


In [ ]:
plot_metrics(history.history['perplexity'], val_metric=history.history['val_perplexity'], metric_name="perplexity", title="perplexity", ylim=50.0)


In [48]:

class OneStep(tf.keras.Model):
  def __init__(self, model, word_from_ids, ids_from_words, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.word_from_ids = word_from_ids
    self.ids_from_words = ids_from_words

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_words(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_words.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function(reduce_retracing=True)
  def expand_dims_if_neccesary(self, input):
    if len(input.shape) < 3:
      input = tf.expand_dims(input, axis=0)
    return input

  @tf.function(reduce_retracing=True)
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_words = preprocess_text(inputs, should_join=False)
    input_ids = self.expand_dims_if_neccesary(self.ids_from_words(input_words))
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_word = self.word_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_word, states

In [49]:
one_step_model = OneStep(model, words_from_ids, ids_from_words)
start = time.time()
states = None
response = tf.constant(['Hi! We are sorry'])
# result = [description]

for n in range(100):
  next_word, states = one_step_model.generate_one_step(response.numpy()[0].decode(), states=states)
  response = tf.concat([response, next_word], axis=0)


result = tf.strings.join(response, separator=" ")
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)


tf.Tensor(b'Hi! We are sorry to for to to for that to for for for for you for to for for you for to for for you for if for for for for for for for for for to to karen for for for to but for to if for for for for for for to for for to about for for for to your for to for for for for for to to erin to our for for for for for that about for for to to for to for to to to for to for for this for about to this for for', shape=(), dtype=string) 

________________________________________________________________________________

Run time: 1.393587589263916
